In [5]:
import torch
from torch.utils.data import DataLoader

import pandas as pd

from models import ShallowNN
from evals import evaluate_mae_with_confidence

features = 197
batch_size = 64
loss_fn = torch.nn.L1Loss()

In [2]:
selected_client_ids = ["0_0", "0_2", "0_3", "0_4", "1_0", "1_1", "1_2", "1_4", "2_0", "2_1", "2_2", "2_3", "2_4", "2_5", "3_0", "3_2","3_3","3_4","3_5"]

client_ids = [f"{i}_{j}" for i in range(4) for j in range(6)]

In [3]:
selected_global_model = ShallowNN(features)
selected_global_model.load_state_dict(torch.load('checkpt/saving_only_selected_clients/epoch_500/_fedl_global_20_25.pth'))

global_model = ShallowNN(features)
global_model.load_state_dict(torch.load('checkpt/saving/epoch_500/_fedl_global_20_25.pth'))

<All keys matched successfully>

In [6]:
selected_eval_list = []
for client in selected_client_ids:
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    
    federated_mae,(fed_lower_band, fed_upper_band), _ = evaluate_mae_with_confidence(selected_global_model,val_loader)
    
    eval_dict = {"Client ID" : client,
                "Federated Average MAE" :round(federated_mae, 4),
                "Federated MAE lower band": round(fed_lower_band, 4),
                "Federated MAE upper band":round(fed_upper_band,4)}
    
    selected_eval_list.append(eval_dict)
    
selected_eval_df = pd.DataFrame.from_dict(selected_eval_list)
selected_eval_df["clients"] = [i for i in range(1, 25) if i not in (2, 6, 10, 12, 20)]

In [7]:
eval_list = []
for client in client_ids:
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    
    federated_mae,(fed_lower_band, fed_upper_band), _ = evaluate_mae_with_confidence(global_model,val_loader)
    
    eval_dict = {"Client ID" : client,
                "Federated Average MAE" :round(federated_mae, 4),
                "Federated MAE lower band": round(fed_lower_band, 4),
                "Federated MAE upper band":round(fed_upper_band,4)}
    
    eval_list.append(eval_dict)
    
eval_df = pd.DataFrame.from_dict(eval_list)
eval_df["clients"] = [i for i in range(1, 25)]

In [9]:
for client in selected_client_ids:
    selected_model_mae = selected_eval_df.loc[selected_eval_df['Client ID'] == client]["Federated Average MAE"].values[0]
    all_model_mae = eval_df.loc[eval_df['Client ID'] == client]["Federated Average MAE"].values[0]
    print(client, selected_model_mae, all_model_mae)

0_0 0.4953 0.4795
0_2 0.3337 0.3645
0_3 0.4222 0.4635
0_4 0.3334 0.3594
1_0 0.506 0.5006
1_1 0.5113 0.5013
1_2 0.4223 0.4415
1_4 0.4419 0.4449
2_0 0.4218 0.422
2_1 0.4472 0.4293
2_2 0.4217 0.4237
2_3 0.4245 0.4309
2_4 0.4163 0.418
2_5 0.4266 0.4372
3_0 0.4993 0.5192
3_2 0.5138 0.5252
3_3 0.5037 0.5184
3_4 0.5088 0.5219
3_5 0.5157 0.5306


In [12]:
selected_model_average_mae = selected_eval_df["Federated Average MAE"].mean()
model_average_mae = eval_df["Federated Average MAE"].mean()

In [15]:
print(f"Average mae of selected model {selected_model_average_mae}")
print(f"Average mae of model {model_average_mae}")

Average mae of selected model 0.4508158564567566
Average mae of model 0.4716208279132843
